In [ ]:
import boto3

def approve_latest_model_package(model_package_group_name, region='us-east-1'):
    # Create a SageMaker client
    sagemaker_client = boto3.client('sagemaker', region_name=region)
    
    try:
        # Fetch the latest model package version
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            SortBy='CreationTime',
            MaxResults=1,
            SortOrder='Descending'  # Get the latest version
        )
        
        if 'ModelPackageSummaryList' in response and len(response['ModelPackageSummaryList']) > 0:
            # Get the latest model package
            latest_model_package = response['ModelPackageSummaryList'][0]
            latest_model_package_arn = latest_model_package['ModelPackageArn']
            
            # Approve the latest model package using ModelPackageArn
            sagemaker_client.update_model_package(
                ModelPackageArn=latest_model_package_arn,
                ModelApprovalStatus='Approved'
            )
            
            print(f"Approved model package {latest_model_package_arn}.")
            return latest_model_package_arn
        else:
            print(f"No model package found in group {model_package_group_name}.")
            return None

    except Exception as e:
        print(f"Error approving model package: {str(e)}")
        return None

# Call the function
approve_latest_model_package('MLOpsAWSModelGroup')


In [ ]:
import boto3

def deploy_model_to_endpoint(model_package_group_name, endpoint_name, region='us-east-1'):
    sagemaker_client = boto3.client('sagemaker', region_name=region)
    
    try:
        # Fetch the latest model package version
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            SortBy='CreationTime',
            MaxResults=1,
            SortOrder='Descending'  # Get the latest version
        )
        
        if 'ModelPackageSummaryList' in response and len(response['ModelPackageSummaryList']) > 0:
            # Get the latest model package ARN
            latest_model_package = response['ModelPackageSummaryList'][0]
            model_package_arn = latest_model_package['ModelPackageArn']
            
            # Get the model artifacts (Model Data) S3 URL using describe_model_package
            model_package_details = sagemaker_client.describe_model_package(
                ModelPackageArn=model_package_arn
            )
            model_data_url = model_package_details['ModelPackageDescription']['InferenceSpecification']['Containers'][0]['ModelDataUrl']
            
            # Ensure you have the correct IAM role ARN
            execution_role_arn = 'arn:aws:iam::767397996001:role/SageMakerMLOps'  # Replace with your role ARN

            # Create a SageMaker model from the approved model package ARN
            model_name = f"{endpoint_name}-model"
            model_response = sagemaker_client.create_model(
                ModelName=model_name,
                PrimaryContainer={
                    'Image': 'container_image_uri',  # Replace with actual container URI
                    'ModelDataUrl': model_data_url  # Point to the S3 model artifacts (Model Data URL)
                },
                ExecutionRoleArn=execution_role_arn  # Correct IAM role ARN
            )
            
            print(f"Model {model_name} created successfully.")
            
            # Create an endpoint configuration
            endpoint_config_name = f"{endpoint_name}-config"
            endpoint_config_response = sagemaker_client.create_endpoint_config(
                EndpointConfigName=endpoint_config_name,
                ProductionVariants=[
                    {
                        'VariantName': 'AllTraffic',
                        'ModelName': model_name,
                        'InitialInstanceCount': 1,
                        'InstanceType': 'ml.m5.large'  # Choose appropriate instance type
                    }
                ]
            )
            
            print(f"Endpoint config {endpoint_config_name} created successfully.")
            
            # Create the endpoint
            endpoint_name = f"{endpoint_name}-endpoint"
            endpoint_response = sagemaker_client.create_endpoint(
                EndpointName=endpoint_name,
                EndpointConfigName=endpoint_config_name
            )
            
            print(f"Endpoint {endpoint_name} created and deploying model...")
            return endpoint_response

        else:
            print(f"No model package found in group {model_package_group_name}.")
            return None

    except Exception as e:
        print(f"Error deploying model to endpoint: {str(e)}")
        return None


# Example Usage
deploy_model_to_endpoint('MLOpsAWSModelGroup', 'my-endpoint')


In [ ]:
import boto3

def deploy_model_to_endpoint(model_package_group_name, endpoint_name, region='us-east-1'):
    sagemaker_client = boto3.client('sagemaker', region_name=region)
    
    try:
        # Fetch the latest model package version
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            SortBy='CreationTime',
            MaxResults=1,
            SortOrder='Descending'  # Get the latest version
        )
        
        if 'ModelPackageSummaryList' in response and len(response['ModelPackageSummaryList']) > 0:
            # Get the latest model package name (not ARN, use ModelPackageName here)
            latest_model_package = response['ModelPackageSummaryList'][0]
            model_package_name = latest_model_package['ModelPackageName']
            
            # Get the model package details to extract the model data S3 URL
            model_package_details = sagemaker_client.describe_model_package(
                ModelPackageName=model_package_name
            )
            
            # Extract model data URL (S3 location of the model artifacts)
            model_data_url = model_package_details['ModelPackageDescription']['InferenceSpecification']['Containers'][0]['ModelDataUrl']
            
            # Ensure you have the correct IAM role ARN
            execution_role_arn = 'arn:aws:iam::767397996001:role/SageMakerMLOps'  # Replace with your role ARN

            # Create a SageMaker model from the approved model package
            model_name = f"{endpoint_name}-model"
            model_response = sagemaker_client.create_model(
                ModelName=model_name,
                PrimaryContainer={
                    'Image': 'container_image_uri',  # Replace with actual container URI
                    'ModelDataUrl': model_data_url  # S3 URL to model artifacts
                },
                ExecutionRoleArn=execution_role_arn  # Correct IAM role ARN
            )
            
            print(f"Model {model_name} created successfully.")
            
            # Create an endpoint configuration
            endpoint_config_name = f"{endpoint_name}-config"
            endpoint_config_response = sagemaker_client.create_endpoint_config(
                EndpointConfigName=endpoint_config_name,
                ProductionVariants=[
                    {
                        'VariantName': 'AllTraffic',
                        'ModelName': model_name,
                        'InitialInstanceCount': 1,
                        'InstanceType': 'ml.m5.large'  # Choose appropriate instance type
                    }
                ]
            )
            
            print(f"Endpoint config {endpoint_config_name} created successfully.")
            
            # Create the endpoint
            endpoint_name = f"{endpoint_name}-endpoint"
            endpoint_response = sagemaker_client.create_endpoint(
                EndpointName=endpoint_name,
                EndpointConfigName=endpoint_config_name
            )
            
            print(f"Endpoint {endpoint_name} created and deploying model...")
            return endpoint_response

        else:
            print(f"No model package found in group {model_package_group_name}.")
            return None

    except Exception as e:
        print(f"Error deploying model to endpoint: {str(e)}")
        return None


# Example Usage
deploy_model_to_endpoint('MLOpsAWSModelGroup', 'my-endpoint')


In [ ]:
import boto3

def deploy_model_to_endpoint(model_package_group_name, endpoint_name, region='us-east-1'):
    sagemaker_client = boto3.client('sagemaker', region_name=region)
    
    try:
        # Fetch the latest model package version
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            SortBy='CreationTime',
            MaxResults=1,
            SortOrder='Descending'  # Get the latest version
        )
        
        if 'ModelPackageSummaryList' in response and len(response['ModelPackageSummaryList']) > 0:
            # Get the latest model package ARN and version
            latest_model_package = response['ModelPackageSummaryList'][0]
            model_package_arn = latest_model_package['ModelPackageArn']
            model_package_version = latest_model_package['ModelPackageVersion']
            
            # Get the model package details to extract the model data S3 URL
            model_package_details = sagemaker_client.describe_model_package(
                ModelPackageArn=model_package_arn  # Use ModelPackageArn here
            )
            
            # Extract model data URL (S3 location of the model artifacts)
            model_data_url = model_package_details['ModelPackageDescription']['InferenceSpecification']['Containers'][0]['ModelDataUrl']
            
            # Ensure you have the correct IAM role ARN
            execution_role_arn = 'arn:aws:iam::767397996001:role/SageMakerMLOps'  # Replace with your role ARN

            # Create a SageMaker model from the approved model package
            model_name = f"{endpoint_name}-model"
            model_response = sagemaker_client.create_model(
                ModelName=model_name,
                PrimaryContainer={
                    'Image': 'container_image_uri',  # Replace with actual container URI
                    'ModelDataUrl': model_data_url  # S3 URL to model artifacts
                },
                ExecutionRoleArn=execution_role_arn  # Correct IAM role ARN
            )
            
            print(f"Model {model_name} created successfully.")
            
            # Create an endpoint configuration
            endpoint_config_name = f"{endpoint_name}-config"
            endpoint_config_response = sagemaker_client.create_endpoint_config(
                EndpointConfigName=endpoint_config_name,
                ProductionVariants=[
                    {
                        'VariantName': 'AllTraffic',
                        'ModelName': model_name,
                        'InitialInstanceCount': 1,
                        'InstanceType': 'ml.m5.large'  # Choose appropriate instance type
                    }
                ]
            )
            
            print(f"Endpoint config {endpoint_config_name} created successfully.")
            
            # Create the endpoint
            endpoint_name = f"{endpoint_name}-endpoint"
            endpoint_response = sagemaker_client.create_endpoint(
                EndpointName=endpoint_name,
                EndpointConfigName=endpoint_config_name
            )
            
            print(f"Endpoint {endpoint_name} created and deploying model...")
            return endpoint_response

        else:
            print(f"No model package found in group {model_package_group_name}.")
            return None

    except Exception as e:
        print(f"Error deploying model to endpoint: {str(e)}")
        return None


# Example Usage
deploy_model_to_endpoint('MLOpsAWSModelGroup', 'my-endpoint')


In [ ]:
import boto3
import time

def deploy_latest_model_version(model_group_name, endpoint_name, execution_role_arn, instance_type='ml.m5.xlarge', instance_count=1):
    """
    Deploy the latest approved model version from a SageMaker Model Group to an endpoint.
    
    Args:
        model_group_name (str): Name of the model group
        endpoint_name (str): Name for the endpoint to create/update
        execution_role_arn (str): ARN of the IAM role for SageMaker execution
        instance_type (str): ML instance type for deployment
        instance_count (int): Number of instances to deploy
    """
    sm_client = boto3.client('sagemaker')
    
    # Get the latest approved model version
    response = sm_client.list_model_packages(
        ModelPackageGroupName=model_group_name,
        ModelApprovalStatus='Approved',
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1
    )
    
    if not response['ModelPackageSummaryList']:
        raise Exception(f"No approved model versions found in group {model_group_name}")
    
    model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
    
    # Create model
    model_name = f"{model_group_name}-{int(time.time())}"
    sm_client.create_model(
        ModelName=model_name,
        ExecutionRoleArn=execution_role_arn,
        PrimaryContainer={
            'ModelPackageName': model_package_arn
        }
    )
    
    # Check if endpoint exists
    try:
        sm_client.describe_endpoint(EndpointName=endpoint_name)
        endpoint_exists = True
    except sm_client.exceptions.ClientError:
        endpoint_exists = False
    
    # Create endpoint configuration
    endpoint_config_name = f"{endpoint_name}-config-{int(time.time())}"
    sm_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InstanceType': instance_type,
            'InitialInstanceCount': instance_count,
            'InitialVariantWeight': 1
        }]
    )
    
    if endpoint_exists:
        # Update existing endpoint
        sm_client.update_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=endpoint_config_name
        )
    else:
        # Create new endpoint
        sm_client.create_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=endpoint_config_name
        )
    
    print(f"Waiting for endpoint {endpoint_name} to be ready...")
    waiter = sm_client.get_waiter('endpoint_in_service')
    waiter.wait(EndpointName=endpoint_name)
    print(f"Endpoint {endpoint_name} is ready!")

# Example usage
if __name__ == "__main__":
    # You need to provide your SageMaker execution role ARN
    execution_role = "arn:aws:iam::767397996001:role/SageMakerMLOps"
    
    deploy_latest_model_version(
        model_group_name='MLOpsAWSModelGroup',
        endpoint_name='my-endpoint',
        execution_role_arn=execution_role,
        instance_type='ml.m5.large',
        instance_count=1
    )

Waiting for endpoint my-endpoint to be ready...
